In [1]:
# Import Neccessary Lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

C:\Users\us953\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# reading csv file
df = pd.read_csv('https://raw.githubusercontent.com/FlipRoboTechnologies/ML-Datasets/main/Medical%20Cost%20Insurance/medical_cost_insurance.csv')
df

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

### Exploratory Data Analysis

In [ ]:
# size of file
df.shape

#### This dataset contains 1599 rows and 12 columns, out of which one column represents the target variable and the remaining eleven columns are independent variables.

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
# As we can see, there are two unique values.
df['sex'].unique()

In [ ]:
# As we can see, there are two unique values.
df['smoker'].unique()

In [ ]:
# As we can see, there are four unique values.
df['region'].unique()

In [ ]:
# As we can see, there are no null values.
df.isna().sum()

In [ ]:
df.info()

In [ ]:
# Representation of null values in a heatmap.
sns.heatmap(df.isnull())

In [ ]:
df.head(10)

In [ ]:
# As we can see, there are no NaN values or any white spaces present.

In [ ]:
df.describe()

In [ ]:
# 1. We observed the data and found that there are no null values present because the count shows the same value
# for all the columns.

# 2. We found that the mean is greater than the median (50th percentile) in the 'charges' column, which is our target column,
# indicating the presence of right skewness.

# 3. Additionally, the difference between the 75th percentile and the maximum value suggests the presence of outliers in these
# columns: age, BMI, children, and charges.

#### Univariate Analysis

In [ ]:
ax = sns.countplot(x= 'charges', data=df)
df['charges'].value_counts()

In [ ]:
# We can easily see that there is no class imbalance in the data.

#### Checking outliers

In [ ]:
plt.figure(figsize=(10,6), facecolor='White')
plotnumber = 1
for col in df.columns:
    if plotnumber <=6:
        ax = plt.subplot(3,2,plotnumber)
        sns.boxplot(df[col], palette='Set2_r')
        plt.xlabel(col, fontsize= 15)
        plt.yticks(rotation=0, fontsize=10)
    plotnumber+=1
plt.tight_layout()

In [ ]:
# As we can see, there are outliers present in BMI and smoker data.

In [ ]:
df.info()

In [ ]:
# Separating categorial and numeric columns.
cate_columns = df.select_dtypes(include=['object']).columns
print(cate_columns)

num_columns = df.select_dtypes(include=[np.number]).columns
print(num_columns)

In [ ]:
# Encoding categorial columns using OrdinalEncodes

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
OE = OrdinalEncoder()
for i in df.columns:
    if df[i].dtype == 'object':
        df[i] = OE.fit_transform(df[i].values.reshape(-1,1))

In [ ]:
df

In [ ]:
df.info()

### checking skewness

In [ ]:
df.skew()

In [ ]:
# As we can see, right skewness is present in the children, smoker, and charges columns, 
# while left skewness is present in the sex and region columns.
# skewness present in category columns, no need to remove it (smoker, sex, region)
# So we need to remove skewness of children and charges

### Checking visualization presentation

In [ ]:
plt.figure(figsize=(10,6), facecolor='White')
plotnumber = 1
for col in df.columns:
    if plotnumber <=8:
        ax = plt.subplot(4,4,plotnumber)
        sns.distplot(df[col], color='m', kde_kws={'shade' : True}, hist=False)
        plt.xlabel(col, fontsize= 15)
        plt.yticks(rotation=0, fontsize=10)
    plotnumber+=1
plt.tight_layout()

##### Removing Skewness

In [ ]:
# Removing the skewness by using Cube root square.
df['children'] = np.cbrt(df['children'])

# Removing the skewness by using Cube root square.
df['charges'] = np.cbrt(df['charges'])

In [ ]:
df.skew()

In [ ]:
plt.figure(figsize=(10,6), facecolor='White')
plotnumber = 1
for col in df.columns:
    if plotnumber <=8:
        ax = plt.subplot(4,4,plotnumber)
        sns.distplot(df[col], color='m', kde_kws={'shade' : True}, hist=False)
        plt.xlabel(col, fontsize= 15)
        plt.yticks(rotation=0, fontsize=10)
    plotnumber+=1
plt.tight_layout()

In [ ]:
df.describe()

In [ ]:
# Removing outliers
from scipy import stats

for col in num_columns:
    z_scores = stats.zscore(df[col])
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3)
    df = df[filtered_entries]

In [ ]:
df.describe()

In [ ]:
# Now Correlation between target variable and independent variable.
cor = df.corr()
cor

In [ ]:
# Visualizing the correlation matrix by plotting heatmap
plt.figure(figsize=(20,15))
sns.heatmap(df.corr(), linewidths=0.1, fmt=".1g", linecolor="black",annot=True,cmap="Blues_r")
plt.yticks(rotation = 0)

plt.show()

#### When two independent features are highly correlated, we can drop one of the columns.

In [ ]:
# We can clearly see that there is no positive correlation with the target value,
# but there is a highly negative correlation with the target value.
cor['charges'].sort_values(ascending=False)

In [ ]:
# Separating feature and label
x = df.drop('charges', axis=1)
y = df['charges']

In [ ]:
# Feature Scaling using Standard Scalarization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)

In [ ]:
df["bmi"]=scaler.fit_transform(df[["bmi"]])

In [ ]:
# Checking Variance Inflation Factor(VIF)
# Checking VIF values to confirm whether multicollinearity exists or not.
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif['VIF Values'] = [variance_inflation_factor(x.values, i) for i in range(len(x.columns))]
vif['Feture'] = x.columns

In [ ]:
vif
# So, we can clearly see that there is no multicollinearity present, so we don't need to drop any columns.

In [ ]:
# Now we can move to the model building
y.value_counts()

## Modelling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=.2, shuffle=True)

In [ ]:
m=RandomForestRegressor()
m.fit(x_train, y_train)

In [ ]:
y_pred=m.predict(x_test)
mean_squared_error(y_test,y_pred)

In [ ]:
print("MAE",mean_absolute_error(y_test,y_pred))

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
m=LinearRegression()
m.fit(x_train, y_train)
y_pred=m.predict(x_train)
mean_squared_error(y_train,y_pred)

In [ ]:
y_pred=m.predict(x_test)
mean_squared_error(y_test,y_pred)

In [ ]:
print("MAE",mean_absolute_error(y_test,y_pred))

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
from sklearn.svm import SVR
m=SVR()
m.fit(x_train, y_train)
y_pred=m.predict(x_train)
mean_squared_error(y_train,y_pred)

In [ ]:
y_pred=m.predict(x_test)
print("MAE",mean_absolute_error(y_test,y_pred))

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
m=KNeighborsRegressor()
m.fit(x_train, y_train)
y_pred=m.predict(x_train)
mean_squared_error(y_train,y_pred)

In [ ]:
y_pred=m.predict(x_test)
print("MAE",mean_absolute_error(y_test,y_pred))

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
random_grid =  {'max_depth': [20, 60, 100, None],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5],
 'n_estimators': [20, 50, 100, 200]}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
best_random = rf_random.best_estimator_
y_pred=best_random.predict(x_test)
print("MAE",mean_absolute_error(y_test,y_pred))

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
# Saving the model using .pkl
import joblib
joblib.dump(best_random, 'insurance.pkl')

In [ ]:
# Predicting the saved model
model = joblib.load('insurance.pkl')
prediction = model.predict(x_test)

In [ ]:
# Confirming
a = np.array(y_test)
df = pd.DataFrame()
df['Predict'] = prediction
df['Original'] = a
df.head(60)